# Single-Qubit Gate


*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline
This tutorial introduces how to generate and optimize pulses for single-qubit gates by Quanlse. The outline of this tutorial is as follows:
- Introduction
- Preparation
- Construct Hamiltonian
- Pulse optimization for Hadamard gate
- Pulse optimization for X gate
- Pulse optimization for Z gate
- Pulse optimization for arbitrary single-qubit gate
- Summary

## Introduction

  In quantum computing, we call an action upon qubit(s) a **quantum gate**. In superconducting quantum circuits, a quantum gate is implemented by applying external microwave pulses and magnetic flux. A single-qubit gate, the focus of this section, can be written as the symbol $U$, which is a $2\times2$ unitary matrix in linear algebra. As shown in the graph below, a single-qubit gate can also be visualized by a vector ($|\psi\rangle$) transformation on what we call a Bloch Sphere. In particular, $|\psi\rangle=\cos(\theta/2)|0\rangle+e^{i\phi}\sin(\theta/2)|1\rangle$ represents the superposition of two quantum states, $|0\rangle$ and $|1\rangle$. Via a single-qubit gate, we can transform a quantum state on the Bloch Sphere \[1\]. 


![bloch sphere](figures/sphere.png)

The following are six commonly used single-qubit gates as well as their operators and matrix representations:

| Gate   |      Operation on Bloch Sphere      |  Operator | Matrix Form | 
|----------|:-------------:|:-:|:-:|
| $X$ |  Rotate $\pi$ about the $x$ axis | $\hat{\sigma}_x$ | $\left(\begin{array}{cc} 0 & 1\\1 & 0\end{array}\right)$ |
| $Y$ |  Rotate $\pi$ about the $y$ axis | $\hat{\sigma}_y$ |$\left(\begin{array}{cc} 0 & -i\\i & 0\end{array}\right)$ |
| $Z$ |  Rotate $\pi$ about the $z$ axis | $\hat{\sigma}_z$ |$\left(\begin{array}{cc} 1 & 0\\0 & -1\end{array}\right)$ |
| $S$ |  Rotate $\pi\over 2$ about the $z$ axis |  |$\left(\begin{array}{cc} 1 & 0\\0 & e^{i\frac{\pi}{2}}\end{array}\right)$ |
| $T$ |  Rotate $\pi\over 4$ about the $z$ axis |  |$\left(\begin{array}{cc} 1 & 0\\0 & e^{i\frac{\pi}{4}}\end{array}\right)$ |
| $H$ | First rotate $\pi$ about the $x$ axis, then rotate $\pi\over 2$ about the $z$ axis |  |$\frac{1}{\sqrt{2}}\left(\begin{array}{cc} 1 & 1\\1 & -1\end{array}\right)$ |

**Single-qubit gate in Quanlse**

Please note that in Quanlse, any arbitrary operation on a single qubit is implemented by the following equation (this representation has a global phase in front and no $R_x$ component):
$$
U(\theta, \phi, \lambda) = e^{i(\phi/2+\lambda/2)} R_z(\phi) R_y(\theta) R_z(\lambda) =
\begin{bmatrix} 
    \cos(\theta/2) & - e^{i\lambda} \sin(\theta/2) \\
    e^{i\phi} \sin(\theta/2) & e^{i(\phi + \lambda)} \cos(\theta/2)
\end{bmatrix} ,
$$
where the $e^{i(\phi/2+\lambda/2)}$ is the global phase of the transformation.

Please refer to our [API documentation](https://quanlse.baidu.com/api/) for more information regarding other types of gates Quanlse supports.

To get the physical intuition of the implementation of single-qubit gates, we will now introduce the two means of implementing single-qubit gates on superconducting qubit:
- **Microwave control** uses capacitive coupling between a resonator and the superconducting qubit (X, Y channel).
- **Frequency tuning** uses a local magnetic field (Z channel) for flux-tunable qubits.

The graph below delineates the X/Y/Z channels of a superconducting qubit:

![X/Y/Z controls for single superconducting qubit](figures/hardware_qubit_control.png)


**Experimental microwave pulse implementation**

To experimentally implement microwave control (X, Y control), we need a local oscillator (LO) to produce a high frequency ($\omega_{\rm LO}$) sinusoidal component combined with a low frequency ($\omega_{\rm AWG}$) arbitrary waveform generator (AWG) to produce a pulse envelope in the form of a Gaussian or tangential function, etc., such that $\omega_{d}=\omega_{\rm LO}\pm\omega_{\rm AWG}$. 

**Experimental flux pulse implementation**

The conventional way of implementing flux control (Z control) is to replace the single Josephson junction with a DC-SQUID, which is a loop consisting of two Josephson junctions. Qubit frequency can be tuned by applying an external magnetic flux perpendicular to the loop.

## Preparation
After you have successfully installed Quanlse, you could run the Quanlse program below following this tutorial. To run this particular tutorial, you would need to import the following packages from Quanlse and other commonly-used Python libraries:

In [ ]:
# Import numpy and math
from numpy import round
from math import pi

# Import the Hamiltonian module
from Quanlse.QHamiltonian import QHamiltonian as QHam

# Import simulator interface on Quanlse Cloud Service
from Quanlse.remoteOptimizer import remoteOptimize1Qubit

# Import related packages
from Quanlse.Utils.Functions import project
from Quanlse.QOperator import duff
from Quanlse.QOperation import FixedGate

## Construct Hamiltonian

Now, we switch our focus to implementing a single-qubit gate using Quanlse. In this example, we will model a system formed by a three-level transmon. Ideally, the non-equidistant energy levels in a transmon allow qubit states to be individually addressed through a frequency-selective driving field. However, the fact that the transmon qubit is weakly anharmonic suggests that the finite bandwidth of the driving field can lead to transitions out of the computational states. This effect is known as **leakage**. In our model, leakage into state $|2\rangle$ - the first level above the computational space - is taken into account by treating the superconducting qubit as a simplified three-level system. The Hamiltonian in the rotating frame can be written as \[2\]:

 $$
 \hat{H}=\alpha_q\lvert2\rangle\langle 2\lvert+\frac{\Omega^x(t)}{2}\left[ \hat{a}^\dagger + \hat{a} \right] + \frac{\Omega^y(t)}{2} i \left[\hat{a}^\dagger - \hat{a}\right]+\Omega^z(t)\hat{a}^{\dagger}\hat{a},
 $$

where $\alpha_q$ is the anharmonicity; $\Omega^x(t)$ is the amplitude of the driving pulse in the X channel; $\Omega^y(t)$ is the amplitude of the driving pulse in the Y channel; and $\Omega^z(t)$ is the amplitude of the flux pulses in the Z channel. The two ladder operators are respectively $\hat{a}^\dagger = |1\rangle\langle 0| + \sqrt{2}|2\rangle\langle 1|$ and $\hat{a} = |0\rangle\langle 1| + \sqrt{2}|1\rangle\langle 2|$.

Quanlse can be used to realize arbitrary single-qubit gates. While Quanlse supports various waveforms' definitions. Here, we take the Gaussian pulse function as an example. The Gaussian pulse function takes the form:

$$
A^{x}(t)=A^{x} e^{-((t-\tau^{x})/2\sigma^{x})^2}, 
$$

$$
A^{y}(t)=A^{y} e^{-((t-\tau^{y})/2\sigma^{y})^2} .
$$

In these equations above, $A^{x}, A^{y}, \tau^{x}, \tau^{y}, \sigma^{x}, \sigma^{y}$ are the parameters to be optimized.
Unlike microwave control, the flux channel's input takes the form of a square wave, i.e. $
A^{z}(t) = A^{z}$, where $A^{z}$ is the parameter to be optimized.

Now, we need to construct the above Hamiltonian using Quanlse. We start by defining some of the basic parameters needed for constructing a Hamiltonian object: the sampling period, the number of qubits in the system as well as the system's energy levels to consider:

In [ ]:
# Sampling period
dt = 0.2

# Number of qubits
qubits = 1

# System energy level
level = 3

Then, we define the anharmonicity of the qubit:

In [ ]:
# Define anharmonicity
qubitArgs = {
    "qubit_anharm": - 0.33 * (2 * pi),  # Anharmonicity of the qubit
}

We create the Hamiltonian object using the initilizer `Qham()`. When initializing, we specify the qubit number, system's energy level number and the sampling period of the AWG(arbitrary wave generator). Then, the users would need to add the anharmonicity term using the function `addDrift()` which takes three arguments: the corresponding operator, the qubit index, and the amplitude of the anharmonicity.

In [ ]:
# Create the Hamiltonian.
ham = QHam(qubits, level, dt)

# Add the drift term(s).
ham.addDrift(duff, 0, coef=qubitArgs["qubit_anharm"] / 2)

## Pulse optimization for Hadamard gate

With the system Hamiltonian created, we can now generate and optimize the pulses to a selected single-qubit gate with desired infidelity (we choose the Hadamard gate as an example) using the following function: `remoteOptimize1Qubit()`. The optimization usually takes a long time to process on local devices, however, we provide a cloud service that could speed up this process significantly. To use Quanlse Cloud Service, the users need to acquire a token from http://quantum-hub.baidu.com and use the following command to submit the job onto Quanlse's server. For this specific example, we can write:

In [ ]:
# Import Define class and set the token
# Please visit http://quantum-hub.baidu.com
from Quanlse import Define
Define.hubToken = ''

# Run the optimization
gateJob, infidelity = remoteOptimize1Qubit(ham, FixedGate.H.getMatrix(), depth=4, targetInfid=0.0001)

In the optimization above, the gate infidelity for performance assessment is defined as ${\rm infid} = 1 - \frac{1}{d}\left|{\rm Tr}[U^\dagger_{\rm goal}P(U)]\right|$, where $U_{\rm goal}$ is the unitary matrix of the target single-qubit gate and $d$ is the dimension of $U_{\rm goal}$; $U$ is the unitary evolution of the system. Note that $P(U)$ in particular describes the evolution projected to the computational subspace.

The function `remoteOptimize1Qubit()` takes in a Hamiltonian object (`ham`), a target unitary matrix (`FixedGate.H.getMatrix()`), the maximum pulse circuit's depth (also maximum pulse number), and a target infidelity. While this demo generates a decent fidelity for the Hadamard gate, we encourage the users to try varying these parameters to get the optimal result.

The `plot()` function allows us to visualize the pulse generated. We can also obtain the projected evolution:

In [ ]:
# Print infidelity and the waveforms
print(f"minimum infidelity: {infidelity}")
gateJob.plot(dark='True')

# Print the evolution process.
result = ham.simulate(job=gateJob)
projectedEvolution = project(result.result[0]["unitary"], qubits, level, 2)
print("Projected evolution:\n", round(projectedEvolution, 2))

`plotWaves()`'s function arguments include a Hamiltonian object (`ham`), the channels to print(`["q0-ctrlx", "q0-ctrly", "q0-ctrlz"]`) and an optional bool parameter `dark`, which enables a dark-themed mode. Moreover, the user can use the `color` parameter to specify colors for individual pulses (the colors will repeat if there are more pulses than colors). Apart from the pulse visualization, we can also obtain the projected evolution shown above.

The following are demonstrations for the X and Z gates, using the exact same system Hamiltonian as above.

## Pulse optimization for X gate

The following code generates and optimizes the pulses for an X gate. Here, we specify the pulse numbers on the X and Y channels by setting the parameter: `xyzPulses=[1, 1, 0]`.

In [ ]:
# Run the optimization
gateJob, infidelity = remoteOptimize1Qubit(ham, FixedGate.X.getMatrix(), depth=4, targetInfid=0.0001)

# Print infidelity and the waveforms
print(f"minimum infidelity: {infidelity}")
gateJob.plot(dark='True')

# Print the evolution process.
result = ham.simulate(job=gateJob)
projectedEvolution = project(result.result[0]["unitary"], qubits, level, 2)
print("Projected evolution:\n", round(projectedEvolution, 2))

## Pulse optimization for Z gate

The following code generates and optimizes the pulses for a Z gate.

In [ ]:
# Run the optimization
gateJob, infidelity = remoteOptimize1Qubit(ham, FixedGate.Z.getMatrix(), depth=4, targetInfid=0.0001)

# Print infidelity and the waveforms
print(f"minimum infidelity: {infidelity}")
gateJob.plot(dark='True')

# Print the evolution process.
result = ham.simulate(job=gateJob)
projectedEvolution = project(result.result[0]["unitary"], qubits, level, 2)
print("Projected evolution:\n", round(projectedEvolution, 2))

## Pulse optimization for arbitrary single-qubit gate

The following code generates and optimizes the pulses for an arbitrary gate, `U(theta=-1.231, phi=1.231, lamda=-1.231)`. Note that the user would need to import `U` from `RotationGate`.

In [ ]:
from Quanlse.QOperation.RotationGate import U

# Define a U3 gate
aGate = U(theta=-1.231, phi=1.231, lamda=-1.231)

# Run the optimization
gateJob, infidelity = remoteOptimize1Qubit(ham, aGate.getMatrix(), depth=4, targetInfid=0.0001)

# Print infidelity and the waveforms
print(f"minimum infidelity: {infidelity}")
gateJob.plot(dark='True')

# Print the evolution process.
result = ham.simulate(job=gateJob)
projectedEvolution = project(result.result[0]["unitary"], qubits, level, 2)
print("Projected evolution:\n", round(projectedEvolution, 2))

## Summary

This tutorial introduces the complete process of generating and optimizing pulses for any single-qubit gate using Quanlse. The users could follow this link [tutorial-single-qubit-gate.ipynb](https://github.com/baidu/Quanlse/blob/main/Tutorial/EN/tutorial-single-qubit.ipynb) to the GitHub page of this Jupyter Notebook document and obtain the relevant code for themselves. The users are encouraged to try parameter values different from this tutorial to obtain the optimal result.

## References
\[1\] [Nielsen, Michael A., and Isaac L. Chuang. Quantum Computation and Quantum Information: 10th Anniversary Edition. Cambridge University Press, 2010.](https://doi.org/10.1017/CBO9780511976667)

\[2\] [Wilhelm, Frank K., et al. "An introduction into optimal control for quantum technologies." *arXiv preprint arXiv:2003.10132* (2020).](https://arxiv.org/abs/2003.10132)